## Task 0

In [1]:
# import liabraries
# sqlite3 helps Python open the sqlite file
# duckdb helps to create a duckdb database
# pandas use to load and move the data

import duckdb
import pandas  as pd
import matplotlib.pyplot as plt 

In [ ]:
# # Open sqlite database to read tables

# # Connnect to sqlite database, bridge btwn Python & sakila.sqlite file
# sqlite_conn = sqlite3.connect(sqlite_path)   

# # Read table names: asks SQLite to show all table names inside this data
# tables_query="SELECT name FROM sqlite_master WHERE type ='table';"

# # pd.read_sql runs the sql query & makes it easy to view the table names.
# tables = pd.read_sql(tables_query, sqlite_conn)

# # Display the list of tables so we know what exists in sqlite database
# tables


,name


### Connect to DuckDB

In [2]:
# Connect to duckdb database

con = duckdb.connect("sakila.duckdb")

## Task  1

#### a) Movies longer than 3-hours (180 min), show the title and its length

In [ ]:
# Select movies longer than 180 minutes
query = """ 
SELECT title, length
FROM film
WHERE length > 180
"""

# Run SQL and convert to pandas dataframe
long_movies = con.execute(query).df()

# Show result
long_movies 

#### b) Movies with the word "love" in the title

In [ ]:
# Find movies that contain 'love' in the title
query = """ 
SELECT title, rating, length, description
FROM film
WHERE LOWER(title) LIKE '%love%'
"""

love_movies = con.execute(query).df()
love_movies

#### c) Movies length statistics
We calculate:
- shortest
- average
- median
- longest

In [ ]:
query = """ 
SELECT
    MIN(length) AS shortest,
    AVG(length) AS average,
    MEDIAN(length) AS median,
    MAX(length) AS longest
FROM film
"""

movie_stats = con.execute(query).df()
movie_stats

#### d) 10 most expensive movies to rent per day

In [ ]:
cost per day = rental_rate / rental_duration

query = """ 
SELECT title,
       rental_rate,
       rental_duration,
       rental_rate / rental_duration AS cost_per_day
FROM film
ORDER BY cost_per_day DESC
LIMIT 10
"""

expensive_movies = con.execute(query).df()
expensive_movies